# Librerías

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import  pywhatkit as kit
from datetime import datetime
from time import sleep
import pandas as pd
import os
import time

In [21]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
service = Service('/usr/bin/chromedriver')

Comedor Universitario Siberia
SELECCIONAR
Comedor ESC AGROTECNICA
SELECCIONAR
Comedor Universitario Área Salud
SELECCIONAR
Comedor Universitario FCEIA
SELECCIONAR
Comedor Universitario Zavalla
SELECCIONAR
Comedor Universitario Casilda
SELECCIONAR
Comedor Universitario Área Centro

In [22]:
cod_comedores = {
    'siberia' : 0,
    'agrotecnia' : 1,
    'salud' : 2,
    'fceia' : 3,
    'zaballa' : 4,
    'casilda' : 5,
    'centro' : 6,
}

In [23]:
cod_menues_salud = {
    'des' : 0,
    'alm_llevar_14' : 1,
    'alm_llevar_13' : 2,
    'alm_llevar_12' : 3,
    'alm_llevar_11' : 4,
    'alm_no_llevar_14' : 5,
    'alm_no_llevar_13' : 6,
    'alm_no_llevar_12' : 7,
    'mer' : 8,
    'cen_llevar' : 9,
    'cen_no_llevar_20' : 10,
    'cen_no_llevar_19.30' : 11,
}

cod_menues_siberia = {
    'des' : 0,
    'alm_llevar_14' : 1,
    'alm_llevar_13' : 2,
    'alm_llevar_12' : 3,
    'alm_llevar_11' : 4,
    'alm_no_llevar_14' : 5,
    'alm_no_llevar_13' : 6,
    'alm_no_llevar_12' : 7,
    'mer' : 8,
    'cen_llevar' : 9,
    'cen_no_llevar_20' : 10,
    'cen_no_llevar_19.30' : 11,
}

cod_menues_fceia = {
    'des' : 0,
    'alm_llevar' : 1,
    'alm_no_llevar_14' : 2,
    'alm_no_llevar_13' : 3,
    'alm_no_llevar_12' : 4,
    'mer' : 5,
    'cen_llevar' : 9,
    'cen_no_llevar_20' : 10,
    'cen_no_llevar_19.30' : 11,
}

In [24]:
op_menu_df = pd.read_excel('./facundo.xlsx',sheet_name='opciones_menu')
op_menu_df['fceia'].to_list().index('des')

0

In [25]:
comedores_df = pd.read_excel('./facundo.xlsx',sheet_name='comedores')
comedores_df['comedores'].to_list().index('siberia')

0

In [26]:
menues_elegidos_df = pd.read_excel('./facundo.xlsx',sheet_name='menues')
menues_elegidos_df['fecha'].dt.day

C:\Users\Usuario\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


0    26
1    26
2    27
3    28
4    29
5    30
Name: fecha, dtype: int32

In [ ]:
class Reserva:
    driver = webdriver.Chrome(options=chrome_options)
    url_login = 'https://comedores.unr.edu.ar/'
    url_reservas = 'https://comedores.unr.edu.ar/comedor-reserva/reservar'
    comedor = None
    menu = None
    u_logueado = False
    
    def __init__(self,u_dni:'str',u_clave:'str',op_eledigas:pd.DataFrame):
        self.u_dni = u_dni
        self.u_clave = u_clave
        self.op_eledigas = op_eledigas
    def cerrar_navegador(self):
        print('se cierra el navegador')
        self.driver.quit()
    
    def loguearse(self):
        # Abrir la página web
        try:
            self.driver.get(self.url_login)
            # Capturar el elemento formulario
            e_form = WebDriverWait(driver=self.driver, timeout=2).until(
                EC.presence_of_element_located(locator=(By.ID, "form-login"))
            )
            print('Formulario de logeo recibido')
            # Capturar los elementos del formulario
            e_input_dni = e_form.find_element(by=By.NAME,value='form-login[dni]')
            e_input_clave = e_form.find_element(by=By.NAME,value='form-login[clave]')
            # Completar el forumalario
            e_input_dni.send_keys('34178096')
            e_input_clave.send_keys('facundo0301')
            # Enviar el formulario
            e_boton = e_form.find_element(by=By.NAME,value='botones[botonEnviar]')
            e_boton.click()
            WebDriverWait(self.driver, 10).until(
                EC.invisibility_of_element_located(locator=(By.ID, "form-login"))
            )
            self.driver.implicitly_wait(200)
            print('Usuario logueado, redireccionando...\n')        
            # Ingresar a la sección de reserva
            self.u_logueado = True
        except Exception as e:
            print(f"Ocurrió un error al visitar la página: {e}.")
            self.cerrar_navegador()
    def ingresar_comedor(self,comedor):
        if not self.u_logueado: 
            print('para elegir un comedor primero hay que loguearse')
            return
        # Abrir la página web
        try:
            self.driver.get(self.url_reservas)
            e_contenedor_comedores = WebDriverWait(driver=self.driver, timeout=2).until(
                EC.presence_of_element_located(locator=(By.CLASS_NAME,'reservar-comedores-contenedor'))
            )
            e_botones_comedores = e_contenedor_comedores.find_elements(by=By.TAG_NAME,value='button')
            n_boton_comedor = comedores_df['comedores'].to_list().index(comedor)
            e_boton_salud = e_botones_comedores[n_boton_comedor]
            # Ingresar al comedor de Salud
            e_boton_salud.click()
            print('Ingreso al comedor',comedor)
            self.comedor = comedor
        except Exception as e:
            print(f"Ocurrió un error al visitar la página: {e}.")
    def buscar_menu(self,menu,dia):
        if self.comedor is None: 
            print('Para reservar un menú primero hay que elegir un comedor')
            return
        try:
            op_reservas = self.driver.find_elements(by=By.CLASS_NAME,value='reservar-servicio')
            n_op_mer = op_menu_df[self.comedor].to_list().index(menu)
            op_mer = op_reservas[n_op_mer]
            op_mer.click()
            self.driver.implicitly_wait(20)
            print(f'Avancemos en reservar el menú {menu} el día {dia}')
            lote_span_n_dia = self.driver.find_elements(by=By.XPATH,value="//span[@data-bind='text: numero']")
            span_n_dia_reserva = lote_span_n_dia[dia-1]
            div_class_calendario_dia = span_n_dia_reserva.find_element(by=By.XPATH,value="../..")
            div_class_calendario_dia_turno = div_class_calendario_dia.find_element(by=By.CLASS_NAME,value="calendario-dia-turno")
            info_enlazada = div_class_calendario_dia_turno.get_attribute(name='data-bind')
            if not info_enlazada:
                print(f'El menú no está disponible: día no habilitado o ya se hizo la reserva\n')
            else:
                div_class_calendario_dia_turno.click()
                wait = WebDriverWait(self.driver, 10)
                div_swal2_container = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "swal2-container")))
                div_swal2_error = self.driver.find_element(By.CLASS_NAME, "swal2-error")
                div_swal2_error_estilo = div_swal2_error.get_attribute('style')
                if div_swal2_error_estilo != 'display: none;': 
                    h2_msg_error = div_swal2_container.find_element(by=By.TAG_NAME,value='h2')
                    msg_error = h2_msg_error.text
                    print(f'Hay problemas con la reserva del menú: {msg_error}\n')
                else:    
                    boton_reservar = self.driver.find_element(By.CSS_SELECTOR, ".swal2-confirm")
                    boton_reservar.click()
                    wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, "swal2-container")))
                    ticket = self.driver.find_element(by=By.CLASS_NAME,value='ticket')
                    print('hay ticket',ticket)
                    print(f'Menú reservado\n')
        except Exception as e:
            print(f"Ocurrió un error al visitar la página: {e}.")
    def buscar_menues(self):
        for _,op_elegida in self.op_eledigas.iterrows():
            comedor_elegido = op_elegida['comedor']
            menu_elegido = op_elegida['menu']
            dia_elegido = op_elegida['fecha'].day
            mes_elegido = op_elegida['fecha'].month
            self.ingresar_comedor(comedor=comedor_elegido)
            self.buscar_menu(menu=menu_elegido,dia=dia_elegido)
        self.cerrar_navegador()    

In [28]:
reserva = Reserva(u_dni='34178096',u_clave='facundo0301',op_eledigas=menues_elegidos_df)
reserva.loguearse()
reserva.buscar_menues()

Formulario de logeo recibido
Usuario logueado, redireccionando...

5
Ingreso al comedor salud
Avancemos en reservar el menú mer el día 26
El menú no está disponible: día no habilitado o ya se hizo la reserva

5
Ingreso al comedor salud
Avancemos en reservar el menú mer el día 26
El menú no está disponible: día no habilitado o ya se hizo la reserva

5
Ingreso al comedor salud
Avancemos en reservar el menú mer el día 27
El menú no está disponible: día no habilitado o ya se hizo la reserva

5
Ingreso al comedor fceia
Avancemos en reservar el menú mer el día 28
El menú no está disponible: día no habilitado o ya se hizo la reserva

5
Ingreso al comedor salud
Avancemos en reservar el menú mer el día 29
Hay problemas con la reserva del menú: No tenés saldo suficiente

5
Ingreso al comedor fceia
Avancemos en reservar el menú mer el día 30
El menú no está disponible: día no habilitado o ya se hizo la reserva

se cierra el navegador
